# Lab 1: Silver Fund Quant Data Module and Returns

In this lab we will:
- Explore how to pull data from the Silver Fund Quant data module.
- Demonstrate the different properties of returns.

## Setup

In order to have a smooth experience with this lab do the following:

### 1. Log into the Fulton Super Computer.

In order to log into you must have an account at [https://rc.byu.edu/](https://rc.byu.edu/) and be added to the `grp_quant` group by Brian Boyer.

It can take some time to get approved so make sure to create an account and reach out to Brian promptly.

### 2. Clone this repo to the desired location (I prefer to have a `Projects` folder where I keep all of my repositories).

Clone the repo by running
```bash
git clone https://github.com/BYUSilverFund/sf-quant-labs.git
```

### 3. Install `uv` (Package Manager)

We use `uv` to create and manage virtual environments.

To install `uv` run

```bash
curl -LsSf https://astral.sh/uv/install.sh | sh
```

Check that `uv` is installed by running

```bash
uv --version
```

If this returns an error you might need to add uv to your path. Run:

```bash
source $HOME/.local/bin/env
```

Restart your terminal for the changes to take effect.

### 4. Create a Virtual Environment

The virtual environment will make it so that we have consistent package and Python versions across all devices.

With `uv` it is really easy to create a virtual environment with synced dependencies.

Just run

```bash
uv sync
```

Activate the environment by running

``` bash
source .venv/bin/activate
```

## Imports

With all of the setup out of the way we will import the necessary Python packages for the lab.

- `sf_quant`: Silver Fund Quant Team package that includes modules for loading data, optimizing portfolios, backtesting, and analyzing performance.
- `datetime`: Native Python library for creating Python `date` types.
- `polars`: Data frame library similar to Pandas but with a much cleaner API and 100x speed ups.

In [2]:
import sf_quant as sf
import polars as pl
import datetime as dt

/home/kylecm11/silver_fund/sf-quant-labs/.venv/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-09-05 12:07:14,932	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Data

Use the following code to pull data for our investment universe from 2024-01-01 to 2024-12-31.

In [9]:
sf.data.get_assets_columns()

'shape: (31, 2)\n┌─────────────────────────────────┬─────────┐\n│ column                          ┆ dtype   │\n│ ---                             ┆ ---     │\n│ str                             ┆ str     │\n╞═════════════════════════════════╪═════════╡\n│ date                            ┆ Date    │\n│ rootid                          ┆ String  │\n│ barrid                          ┆ String  │\n│ issuerid                        ┆ String  │\n│ instrument                      ┆ String  │\n│ name                            ┆ String  │\n│ cusip                           ┆ String  │\n│ ticker                          ┆ String  │\n│ price                           ┆ Float64 │\n│ return                          ┆ Float64 │\n│ specific_return                 ┆ Float64 │\n│ market_cap                      ┆ Float64 │\n│ price_source                    ┆ String  │\n│ currency                        ┆ String  │\n│ iso_country_code                ┆ String  │\n│ iso_currency_code               ┆ String 

In [3]:
start = dt.date(2024, 1, 1)
end = dt.date(2024, 12, 31)

columns = [
    'date',
    'barrid',
    'return',
    'price'
]

df = sf.data.load_assets(
    start=start,
    end=end,
    in_universe=True,
    columns=columns
)

df

date,barrid,return,price
date,str,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,7.87
2024-01-03,"""USA06Z1""",-1.2071,7.775
2024-01-04,"""USA06Z1""",-0.1929,7.76
2024-01-05,"""USA06Z1""",0.5155,7.8
2024-01-08,"""USA06Z1""",5.3846,8.22
…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,70.58
2024-12-26,"""USBQOR1""",4.293,73.61
2024-12-27,"""USBQOR1""",-5.108,69.85


In [10]:
df = df.with_columns(pl.col("price").pct_change().alias("price_change"))
df

date,barrid,return,price,price_diff,price_change
date,str,f64,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,7.87,null,null
2024-01-03,"""USA06Z1""",-1.2071,7.775,-0.095,-0.012071
2024-01-04,"""USA06Z1""",-0.1929,7.76,-0.015,-0.001929
2024-01-05,"""USA06Z1""",0.5155,7.8,0.04,0.005155
2024-01-08,"""USA06Z1""",5.3846,8.22,0.42,0.053846
…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,70.58,1.78,0.025872
2024-12-26,"""USBQOR1""",4.293,73.61,3.03,0.04293
2024-12-27,"""USBQOR1""",-5.108,69.85,-3.76,-0.05108


## Log returns

### Instructions
1. Compute the log returns for each asset.
2. Compute the cummulative log returns for each asset.
3. Run the assertion cell to make sure you're results are correct.

Make sure to sort prior to computing time series metrics and use `.over()` apply the computation in groups.

Log returns have the nice property of being additive. Use this to your advantage!

In [11]:
def task_compute_log_returns(df: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the log returns for each security and date combo.

    Args:
        df (pl.DataFrame): Data frame containing columns date, barrid, and return

    Returns:
        pl.DataFrame: Data frame containing columns date, barrid, return, and log_return
    """

    # TODO: Finish this function
    df = df.sort(by=["barrid", "date"])

    df = df.with_columns(
        (pl.col("return") / 100 + 1).log().alias("log_return")
    )

    return df

df_log = task_compute_log_returns(df)

df_log

date,barrid,return,price,price_diff,price_change,log_return
date,str,f64,f64,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,7.87,null,null,-0.108279
2024-01-03,"""USA06Z1""",-1.2071,7.775,-0.095,-0.012071,-0.012144
2024-01-04,"""USA06Z1""",-0.1929,7.76,-0.015,-0.001929,-0.001931
2024-01-05,"""USA06Z1""",0.5155,7.8,0.04,0.005155,0.005142
2024-01-08,"""USA06Z1""",5.3846,8.22,0.42,0.053846,0.052446
…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,70.58,1.78,0.025872,0.025543
2024-12-26,"""USBQOR1""",4.293,73.61,3.03,0.04293,0.042034
2024-12-27,"""USBQOR1""",-5.108,69.85,-3.76,-0.05108,-0.052431


In [12]:
def task_compute_cumulative_log_returns(df_log: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the cummulative log returns for each security and date combo.

    Args:
        df_log (pl.DataFrame): Data frame containing columns date, barrid, return, and log_return

    Returns:
        pl.DataFrame: Data frame containing columns date, barrid, return, log_return, and cumulative_log_return
    """

    # TODO: Finish this function

    df = df_log.sort(by=["barrid", "date"])
    df = df.with_columns(pl.col("log_return").cum_sum().over("barrid").alias("cumulative_log_return"))

    return df

df_cum_log = task_compute_cumulative_log_returns(df_log)

df_cum_log

date,barrid,return,price,price_diff,price_change,log_return,cumulative_log_return
date,str,f64,f64,f64,f64,f64,f64
2024-01-02,"""USA06Z1""",-10.2623,7.87,null,null,-0.108279,-0.108279
2024-01-03,"""USA06Z1""",-1.2071,7.775,-0.095,-0.012071,-0.012144,-0.120424
2024-01-04,"""USA06Z1""",-0.1929,7.76,-0.015,-0.001929,-0.001931,-0.122355
2024-01-05,"""USA06Z1""",0.5155,7.8,0.04,0.005155,0.005142,-0.117213
2024-01-08,"""USA06Z1""",5.3846,8.22,0.42,0.053846,0.052446,-0.064766
…,…,…,…,…,…,…,…
2024-12-24,"""USBQOR1""",2.5872,70.58,1.78,0.025872,0.025543,0.087605
2024-12-26,"""USBQOR1""",4.293,73.61,3.03,0.04293,0.042034,0.129639
2024-12-27,"""USBQOR1""",-5.108,69.85,-3.76,-0.05108,-0.052431,0.077208


In [13]:
df_cum_log['cumulative_log_return'].max()

2.8475532093020552

In [ ]:
assert df_cum_log['cumulative_log_return'].max() == 2.8475532093020557

## Compounded Returns

### Instructions

1. Compute the cumulative compounded returns for each asset.
2. Run the assertion to check that your results are correct.

In [ ]:
def task_compute_cumulative_compounded_returns(df_cum_log: pl.DataFrame) -> pl.DataFrame:
    """
    Compute the cumulative compounded returns for each security.

    Args:
        df_cum_log (pl.DataFrame): Data frame containing columns date, barrid, return, log_return, and cumulative_log_return

    Returns:
        pl.DataFrame: Data frame containing columns date, barrid, return, log_return, cumulative_log_return, and cumulative_compouned_return
    """

    # TODO: Finish this function

    pass

df_cum_comp = task_compute_cumulative_compounded_returns(df_cum_log)

df_cum_comp

In [ ]:
assert df_cum_comp['cumulative_compounded_return'].max() == 16.245533963705515

## Exponentiation

Note that the max cumulative log return is different from the cumulative compounded return.

Why is that?

The answer is that the cumulative log return is still in log space!

### Instructions

1. Exponentiate the cumulative log returns to put them back into the original space.
2. Check that the exponentiated returns match the cumulative compounded returns.

In [ ]:
def task_exponentiate_returns(df_cum_comp: pl.DataFrame) -> pl.DataFrame:
    """
    Exponentiate the cumulative log returns.

    Args:
        df_cum_comp: Data frame containing date, barrid, return, log_return, cumulative_log_return, and cumulative_compouned_return.

    Returns:
        pl.DataFrame: Data frame containing all previous columns plus exponentiated_returns
    """

    # TODO: Finish this function

    pass

df_exp = task_exponentiate_returns(df_cum_comp)

df_exp

In [ ]:
assert df_exp['cumulative_compounded_return'].max() == df_exp['exponentiated_return'].max()

In [10]:
import numpy as np


In [16]:
w = np.array([.4, .6])
r = np.array([.10, .05])

Cov = np.array([[0.09, 0.025], [0.025, 0.04]])

np.sqrt(w.T @ Cov @ w)

np.float64(0.20199009876724155)

In [18]:
w.T @ r / np.sqrt(w.T @ Cov @ w)


np.float64(0.34655164004183603)